In [1]:
%pwd

'/home/csio/rahul/Credit_Card_Behaviour_Predicition/notebook'

In [2]:
import os
os.chdir('../')
%pwd

'/home/csio/rahul/Credit_Card_Behaviour_Predicition'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path: Path

In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])
        data_tranformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_tranformation_config

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
import joblib
import os
from src import logger

In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def remove_highly_correlated_features(self, df, threshold=0.95):
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
        return df.drop(columns=to_drop)
    
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        X = data.drop(['account_number', 'bad_flag'], axis=1)
        y = data['bad_flag']
        missing_pct = (X.isnull().sum() / len(X)) * 100
        cols_to_drop = missing_pct[missing_pct > 70].index
        X_cleaned = X.drop(columns=cols_to_drop)
        numeric_cols = X_cleaned.select_dtypes(include=['float64', 'int64']).columns
        X_cleaned[numeric_cols] = X_cleaned[numeric_cols].fillna(X_cleaned[numeric_cols].median())
        X_cleaned = self.remove_highly_correlated_features(X_cleaned, threshold=0.95)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_cleaned)
        X_scaled = pd.DataFrame(X_scaled, columns=X_cleaned.columns)
        final_df = pd.concat([X_scaled, y], axis=1)
        train, test = train_test_split(final_df, test_size=0.25, random_state=42, stratify=y)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        joblib.dump(scaler, os.path.join(self.config.root_dir, "scaler.joblib"))
        
        logger.info("Completed data transformation")
        logger.info(f"Training set shape: {train.shape}")
        logger.info(f"Test set shape: {test.shape}")
        
        return train, test

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-01-13 15:01:59,769: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-13 15:01:59,893: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-13 15:02:00,132: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-13 15:02:00,135: INFO: common: created directory at: artifacts]
[2025-01-13 15:02:00,138: INFO: common: created directory at: artifacts/data_transformation]
[2025-01-13 15:11:15,271: INFO: 3626637720: Completed data transformation]
[2025-01-13 15:11:16,261: INFO: 3626637720: Training set shape: (72604, 1017)]
[2025-01-13 15:11:16,361: INFO: 3626637720: Test set shape: (24202, 1017)]
